RNN, LSTM 활용 텍스트 생성하기<br>
참고자료: https://wikidocs.net/45101

사용할 데이터는 뉴욕 타임즈 기사의 제목입니다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드 합니다.<br>

파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [1]:
from google.colab import files
myfile=files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018 (2).csv


In [2]:
import io
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv(io.BytesIO(myfile['ArticlesApril2018.csv']))

In [4]:
data

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,5ae82c93068401528a2ab969,1004,By CLAIRE CAIN MILLER,article,This Common Question Reinforces the Gender Pay...,"['Discrimination', 'Wages and Salaries', 'Labo...",68,Upshot,3,2018-05-01 09:00:01,Unknown,Several states and cities have ordered employe...,The New York Times,News,https://www.nytimes.com/2018/05/01/upshot/how-...
1320,5ae82c95068401528a2ab96b,1043,By TRACY J. GATES,article,"Anna, Llama and Me","['Friendship', 'Dewdney, Anna', 'Writing and W...",65,Well,0,2018-05-01 09:00:02,Family,"The beginning, middle and end of a picture boo...",The New York Times,News,https://www.nytimes.com/2018/05/01/well/family...
1321,5ae82c9d068401528a2ab96d,659,Interview by AUDIE CORNISH,article,Gen. Michael Hayden Has One Regret: Russia,"['Classified Information and State Secrets', '...",66,Magazine,70,2018-05-01 09:00:06,Unknown,"The former N.S.A. and C.I.A. chief on Trump, S...",The New York Times,News,https://www.nytimes.com/2018/05/01/magazine/ge...
1322,5ae82c9f068401528a2ab96f,1155,By JASON ROBERT BROWN,article,There Is Nothin’ Like a Tune,"['Books and Literature', 'Purdum, Todd S', 'Th...",68,BookReview,17,2018-05-01 09:00:07,Book Review,"In “Something Wonderful,” Todd S. Purdum analy...",The New York Times,Review,https://www.nytimes.com/2018/05/01/books/revie...


In [5]:
print(data.columns)

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


headline을 사용, Unknown 데이터 제거

In [6]:
headline=[]
headline.extend(list(data.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [7]:
headline=[n for n in headline if n!="Unknown"]
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

데이터 전처리(1)-구두점 제거

In [8]:
def delete_punctuation(sentence):
  return ''.join(letter for letter in sentence if letter not in string.punctuation)
#하나의 문장을 들어오고 for n in sentence에서 알파벳 하나 하나에 대해서 체크

In [9]:
import string
headline=[delete_punctuation(n) for n in headline]
headline[:5]

['Former NFL Cheerleaders’ Settlement Offer 1 and a Meeting With Goodell',
 'EPA to Unveil a New Rule Its Effect Less Science in Policymaking',
 'The New Noma Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for SelfExpression']

데이터 전처리(2)-소문자로 변경

In [10]:
def change_lower(sentence):
  return sentence.lower()

In [11]:
headline=[change_lower(n) for n in headline]
headline[:5]

['former nfl cheerleaders’ settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

데이터 정수 인코딩

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(headline)
vocab_size=len(tokenizer.word_index)+1

다음 단어 예측을 하여 텍스트를 생성하기 때문에 각 문장을 여러 줄로 분해<br>
예시)<br>
나는 말을 타고 달린다<br>
[나는 말을]<br>
[나는 말을 타고]<br>
[나는 말을 타고 달린다]<br>
또한, 이렇게 분해된 텍스트는 정수 인코딩하여 저장

In [14]:
encoded=tokenizer.texts_to_sequences(headline)
print(encoded[:5])

[[95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102], [96, 3, 1103, 2, 14, 573, 85, 1104, 366, 367, 5, 1105], [1, 14, 1106, 1107], [16, 2, 574, 4, 210, 368, 1108, 2, 369, 575], [9, 162, 2, 130, 6, 1109]]


In [15]:
sequences=[]
for line in encoded:
  for i in range(1, len(line)):
    sequences.append(line[:i+1])
sequences[:12]

[[95, 263],
 [95, 263, 1100],
 [95, 263, 1100, 1101],
 [95, 263, 1100, 1101, 572],
 [95, 263, 1100, 1101, 572, 50],
 [95, 263, 1100, 1101, 572, 50, 7],
 [95, 263, 1100, 1101, 572, 50, 7, 2],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10],
 [95, 263, 1100, 1101, 572, 50, 7, 2, 365, 10, 1102],
 [96, 3],
 [96, 3, 1103]]

데이터 패딩화

In [16]:
max_len=max(len(l) for l in sequences)
print(max_len)

24


In [17]:
sequences=pad_sequences(sequences, maxlen=max_len)
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   95  263]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   95  263 1100]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   95  263 1100 1101]]


데이터 레이블 분리<br>
다음 단어 예측 학습이기 때문에
sequences의 각 데이터에 대해서 가장 오른쪽 데이터가 데이터 레이블이 된다

In [18]:
sequences=np.array(sequences)
sequences.shape

(7809, 24)

In [19]:
sequences[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,  95, 263], dtype=int32)

In [20]:
X_data=sequences[:, :max_len-1]
Y_data=sequences[:, max_len-1]
print(X_data[:3])
print(Y_data[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   95]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   95  263]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   95  263 1100]]
[ 263 1100 1101]


X_data의 경우, 단어이기 때문에, 학습시 Embedding layer를 사용하기 때문에 one-hot encoding 하지 않는다<br>
Y_data의 경우, 마지막에 softmax를 통해서 오차를 비교하기 때문에 one-hot encoding을 실행한다.

In [21]:
from tensorflow.keras.utils import to_categorical
Y_data=to_categorical(Y_data, vocab_size)

In [22]:
Y_data[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

RNN 응용 LSTM 학습 모델 구성

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [24]:
model=Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

모델 학습

In [25]:
model.fit(X_data, Y_data, epochs=200, verbose=2)

Epoch 1/200
245/245 - 5s - loss: 7.6732 - accuracy: 0.0257
Epoch 2/200
245/245 - 2s - loss: 7.0801 - accuracy: 0.0347
Epoch 3/200
245/245 - 2s - loss: 6.8758 - accuracy: 0.0464
Epoch 4/200
245/245 - 2s - loss: 6.7051 - accuracy: 0.0478
Epoch 5/200
245/245 - 2s - loss: 6.5117 - accuracy: 0.0507
Epoch 6/200
245/245 - 2s - loss: 6.2790 - accuracy: 0.0589
Epoch 7/200
245/245 - 2s - loss: 6.0266 - accuracy: 0.0695
Epoch 8/200
245/245 - 2s - loss: 5.7736 - accuracy: 0.0793
Epoch 9/200
245/245 - 2s - loss: 5.5207 - accuracy: 0.0923
Epoch 10/200
245/245 - 2s - loss: 5.2707 - accuracy: 0.1033
Epoch 11/200
245/245 - 2s - loss: 5.0323 - accuracy: 0.1146
Epoch 12/200
245/245 - 2s - loss: 4.7934 - accuracy: 0.1357
Epoch 13/200
245/245 - 2s - loss: 4.5684 - accuracy: 0.1575
Epoch 14/200
245/245 - 2s - loss: 4.3396 - accuracy: 0.1900
Epoch 15/200
245/245 - 2s - loss: 4.1147 - accuracy: 0.2269
Epoch 16/200
245/245 - 2s - loss: 3.8911 - accuracy: 0.2706
Epoch 17/200
245/245 - 2s - loss: 3.6679 - accura

학습된 모델을 바탕으로 문장 생성<br>
가령 10개의 단어로 된 문장을 만든다고 한다면 첫번째 단어만 주어지고 학습된 모델이 뒤에 붙을 단어들을 알려주어 문장을 완성하는 방법 

In [26]:
def sentence_generation(current_word, letter_num): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(letter_num): # n번 반복
        encoded = tokenizer.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len) # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in tokenizer.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [27]:
print(sentence_generation('he', 10))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


he inhaled mustard agent in iraq but the navy denied him
